In [1]:
import sys
sys.path.append('/mnt/d/ariel2/code/core/')
import kaggle_support as kgs
import importlib
import matplotlib.pyplot as plt
import numpy as np
import ariel_load
import copy
import tqdm
kgs.sanity_checks_active = True
kgs.sanity_checks_without_errors = False

local


In [2]:
train_data = kgs.load_all_train_data()
test_data = kgs.load_all_test_data()
len(train_data), len(test_data)

(1100, 1)

In [4]:
loaders = ariel_load.default_loaders()
for d in tqdm.tqdm(train_data):
    dd = copy.deepcopy(d)
    for t in dd.transits:
        t.load_to_step(2,dd,loaders)
    kgs.print_sanity_checks()

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.94s/it]

gain [0.4369, 0.4369] [0.4, 0.5] 1
offset [-1000.0, -1000.0] [-1001, -999] 2
ratio_hot [0.0, 0.0107421875] [0, 0.015] 3
ratio_dead [0.0018838652482269503, 0.0029296875] [0, 0.005] 4
linear_corr_impact [4832.30513633661, 6306.7908586711565] [0, 50000] 9
dark_min [0.0004378425539543764, 0.002066195137730647] [-0.0015, 0.004] 5
dark_max [0.01163899428117778, 0.013298721036968861] [0.0, 0.02] 6
flat_min [0.8617695468230018, 0.9478742031905644] [0.7, 1.1] 7
flat_max [1.0383349568133693, 1.0605497114876032] [0.9, 1.2] 8


In [ ]:
kgs.print_sanity_checks()